In [1]:
from symred.symplectic import ObservableGraph

In [2]:
from symred.chem import MoleculeBuilder
from symred.projection import QubitTapering, CS_VQE
from symred.symplectic import QuantumState
import json

In [3]:
with open('data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)

In [4]:
# Set molecule parameters
speciesname = 'Be_STO-3G_SINGLET'
#print(speciesname in scoring_data)
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms  = mol_data['atoms']
coords = mol_data['coords']
basis  = mol_data['basis'] #'6-31g' #'cc-pVDZ' #
spin   = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis)

Molecule geometry:
Be	0	0	0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -14.351880476202023
MP2 energy:  -14.376238850841515
CCSD energy: -14.403650751752787
FCI energy:  -14.403655108067682


Number of qubits: 10


In [5]:
taper_hamiltonian = QubitTapering(molecule.H_q)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand may be rotated via a sequence of Clifford pi/2 rotations\n')
print(taper_hamiltonian.stabilizers.stabilizer_rotations)
print('\nonto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
print(f'\nThe symetry sector corresponding with the reference {hf_array} is {taper_hamiltonian.stabilizers.coeff_vec}')
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
#sor_tap = taper_hamiltonian.taper_it(aux_operator=sor_ham, ref_state=hf_array)
#sor_tap.coeff_vec/=np.linalg.norm(sor_tap.coeff_vec)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
n_taper = taper_hamiltonian.n_taper
tapered_qubits   = taper_hamiltonian.stab_qubit_indices
untapered_qubits = taper_hamiltonian.free_qubit_indices
hf_tapered = taper_hamiltonian.tapered_ref_state

print(f'The tapered Hartree-Fock state is', QuantumState([hf_tapered]))

We are able to taper 5 qubits from the Hamiltonian.

The symmetry generators are

1.0000000000+0.0000000000j ZIZIIZIZIZ +
1.0000000000+0.0000000000j IZIZIZIZIZ +
1.0000000000+0.0000000000j IIIIZZIIII +
1.0000000000+0.0000000000j IIIIIIZZII +
1.0000000000+0.0000000000j IIIIIIIIZZ

and may be rotated via a sequence of Clifford pi/2 rotations

[('IIIIYZIIII', None), ('IIIIIIYZII', None), ('IIIIIIIIYZ', None), ('YIZIIZIZIZ', None), ('IYIZIZIZIZ', None)]

onto the single-qubit Pauli operators

-1.0000000000+0.0000000000j XIIIIIIIII +
-1.0000000000+0.0000000000j IXIIIIIIII +
-1.0000000000+0.0000000000j IIIIXIIIII +
-1.0000000000+0.0000000000j IIIIIIXIII +
-1.0000000000+0.0000000000j IIIIIIIIXI

The symetry sector corresponding with the reference [1 1 1 1 0 0 0 0 0 0] is [1 1 1 1 1]
The tapered Hartree-Fock state is  1.0000000000 |11000>


In [6]:
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap)

In [7]:
print(cs_vqe.noncontextual_operator)

-8.6720073786+0.0000000000j IIIII +
-0.0597715896+0.0000000000j XIIII +
-0.0466916923+0.0000000000j ZIIII +
-0.0466916923+0.0000000000j IZIII +
0.0380616241-0.0000000000j XZIII +
0.6698678645+0.0000000000j ZZIII +
-0.2968649525+0.0000000000j IIZII +
0.1830962071+0.0000000000j ZIZII +
0.1830962071+0.0000000000j IZZII +
0.0096106137+0.0000000000j XZZII +
-0.2968649525+0.0000000000j IIIZI +
0.1830962071+0.0000000000j ZIIZI +
0.1830962071+0.0000000000j IZIZI +
0.0096106137+0.0000000000j XZIZI +
0.3892355930+0.0000000000j IIZZI +
0.2807146486+0.0000000000j ZIZZI +
0.2807146486+0.0000000000j IZZZI +
-0.0096106137+0.0000000000j XZZZI +
-0.2968649525+0.0000000000j IIIIZ +
0.1830962071+0.0000000000j ZIIIZ +
0.1830962071+0.0000000000j IZIIZ +
0.0096106137+0.0000000000j XZIIZ +
0.3892355930+0.0000000000j IIZIZ +
0.2807146486+0.0000000000j ZIZIZ +
0.2807146486+0.0000000000j IZZIZ +
-0.0096106137+0.0000000000j XZZIZ +
0.3892355930+0.0000000000j IIIZZ +
0.2807146486+0.0000000000j ZIIZZ +
0.280714648

In [8]:
H_graph = ObservableGraph(ham_tap.symp_matrix, ham_tap.coeff_vec)

In [9]:
C_cliques = H_graph.clique_cover(clique_relation='C', colouring_strategy='largest_first')

In [10]:
C0 = C_cliques[0]
C1 = C_cliques[2]

C_union = C0+C1
C_graph = ObservableGraph(C_union.symp_matrix, C_union.coeff_vec)
print(C0)
print()
print(C1)

AC_cliques = C_graph.clique_cover(clique_relation='AC', colouring_strategy='largest_first')

0.2634692003+0.0000000000j IIZZZ +
-0.0597715896+0.0000000000j XIIII +
0.0597715896+0.0000000000j IXIII +
0.0597715896+0.0000000000j XIZZZ +
-0.0597715896+0.0000000000j IXZZZ +
0.0121246896+0.0000000000j IIXXI +
0.0121246896+0.0000000000j IIXIX +
0.0121246896+0.0000000000j IIIXX +
-8.6720073786+0.0000000000j IIIII

0.6698678645+0.0000000000j ZZIII +
-0.0221489539+0.0000000000j YYIII +
-0.0038997696+0.0000000000j ZZXII +
0.0217022449-0.0000000000j YYXII +
-0.0038997696+0.0000000000j ZZIXI +
0.0217022449-0.0000000000j YYIXI +
-0.0038997696+0.0000000000j ZZIIX +
0.0217022449-0.0000000000j YYIIX


In [11]:
print(AC_cliques[1])

0.0121246896+0.0000000000j IIXXI


In [13]:
import numpy as np
from symred.symplectic import StabilizerOp, find_symmetry_basis

random_Z_block = np.random.randint(0,2,(3, 5))
S = StabilizerOp(np.hstack([np.zeros_like(random_Z_block), random_Z_block]), np.ones(3))
print(S)

1.0000000000+0.0000000000j ZZZZZ +
1.0000000000+0.0000000000j ZZIZI +
1.0000000000+0.0000000000j IZZIZ


In [14]:
S.update_sector(hf_tapered)

In [15]:
print(S)

1.0000000000 ZZZZZ +
1.0000000000 ZZIZI +
-1.0000000000 IZZIZ


In [16]:
S.target_sqp = 'X'
print(S.rotate_onto_single_qubit_paulis())

-1.0000000000+0.0000000000j XIIII +
-1.0000000000+0.0000000000j IXIII +
1.0000000000-0.0000000000j IIXII


In [17]:
S.target_sqp = 'X'
print(S.rotate_onto_single_qubit_paulis())

-1.0000000000+0.0000000000j XIIII +
-1.0000000000+0.0000000000j IXIII +
1.0000000000-0.0000000000j IIXII
